## Datasets init

In [88]:
dataset_path = "/kaggle/input/part-of-coco-dataset/"

image_files = os.listdir(dataset_path)

# Вывод первых 5 файлов
print("Примеры изображений:", image_files[:5])

Примеры изображений: ['000000001739.jpg', '000000001993.jpg', '000000001425.jpg', '000000000974.jpg', '000000006870.jpg']


In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hadiepratamatulili/anime-vs-cartoon-vs-human")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/anime-vs-cartoon-vs-human


## CLIP init

In [1]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-fqqutjh8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-fqqutjh8
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import clip
from PIL import Image
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoProcessor, AutoModelForCausalLM
import numpy as np
import time

2025-06-25 02:33:03.576991: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750818783.831301      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750818783.902652      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Модель CLIP для оценки
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 51.4MiB/s]


## Florence-2-Large

In [4]:
# Модель для генерации описаний
model_id = 'microsoft/Florence-2-large'
gen_model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype='auto').eval().to(device)
gen_processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


...

processing_florence2.py:   0%|          | 0.00/48.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
DATASETS = {
    "coco": 
        {"name": "coco_part", "dataset_dir": "/kaggle/input/part-of-coco-dataset/"},
    'anime': 
        {"name": "anime", "dataset_dir": "/kaggle/input/anime-vs-cartoon-vs-human/Data/anime"},
    'cartoon':
        {"name": "cartoon", "dataset_dir": "/kaggle/input/anime-vs-cartoon-vs-human/Data/cartoon"},
    'human':
        {"name": "human", "dataset_dir": "/kaggle/input/anime-vs-cartoon-vs-human/Data/human"}
}

SAMPLES_PER_DATASET = 100

In [7]:
def generate_caption(image):
    prompt = "<CAPTION>"
    inputs = gen_processor(text=prompt, images=image, return_tensors="pt").to(device)

    # Если вход содержит pixel_values, приводим их к нужному типу (только их)
    if 'pixel_values' in inputs:
        inputs['pixel_values'] = inputs['pixel_values'].to(dtype=next(gen_model.parameters()).dtype)

    with torch.no_grad():
        outputs = gen_model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=300,
            num_beams=1,
            do_sample=False,
            early_stopping=True
        )

    caption = gen_processor.batch_decode(outputs, skip_special_tokens=True)[0]
    return caption

In [8]:
def compute_clip_score_with_generated(model, clip_preprocess, dataset_dirs, dataset_name="custom_dataset"):
    """
    dataset_dirs: str или dict
    Если str — обрабатывается как один датасет
    Если dict — ключи это подкатегории, значения — пути к папкам
    """

    if isinstance(dataset_dirs, str):
        # Обычный датасет (без категорий)
        datasets_to_process = {dataset_name: dataset_dirs}
    else:
        # Датасет с категориями (например, anime/cartoon/human)
        datasets_to_process = dataset_dirs

    print(f"\nEvaluating CLIP Score on {dataset_name} with generated captions...")

    scores = []
    generation_times = []

    samples_per_category = SAMPLES_PER_DATASET

    for category, path in datasets_to_process.items():
        print(f"\nProcessing category: {category}")

        dataset = load_dataset('imagefolder', data_dir=path)
        dataset = dataset['train']
        dataset = dataset.shuffle(seed=42).select(range(min(samples_per_category, len(dataset))))

        image_column = "image" if "image" in dataset.features else "IMG"

        for example in tqdm(dataset, desc=f"Processing {category}"):
            image = example[image_column]
            if isinstance(image, str):
                image = Image.open(image).convert("RGB")
            else:
                image = image.convert("RGB")

            # Засекаем время генерации
            start_time = time.time()
            generated_caption = generate_caption(image)
            end_time = time.time()

            generation_times.append(end_time - start_time)

            image_input = clip_preprocess(image).unsqueeze(0).to(device)
            text_input = clip.tokenize([generated_caption]).to(device)

            with torch.no_grad():
                image_features = model.encode_image(image_input)
                text_features = model.encode_text(text_input)

                image_features /= image_features.norm(dim=-1, keepdim=True)
                text_features /= text_features.norm(dim=-1, keepdim=True)

                similarity = (image_features @ text_features.T).item()
                scores.append(similarity)

    average_score = np.mean(scores)
    average_time = np.mean(generation_times)

    print(f"\nCLIP Score on {dataset_name} (generated captions): {average_score:.4f}")
    print(f"Average caption generation time: {average_time:.4f} seconds")

    return average_score, average_time

In [34]:
if __name__ == "__main__":
    results = {}
    for dataset_name, dataset_info in DATASETS.items():
        score, average_time = compute_clip_score_with_generated(clip_model, clip_preprocess, dataset_dirs=dataset_info['dataset_dir'], dataset_name=dataset_info['name'])
        results[dataset_name] = (score, average_time)

    print("\nFinal CLIP Scores:")
    for dataset_name, result in results.items():
        print(f"CLIP Score on dataset {dataset_name}: {result[0]:.4f}")
        print(f'Average time per image on dataset {dataset_name}: {result[1]:.4f}')


Evaluating CLIP Score on coco_part with generated captions...

Processing category: coco_part


Resolving data files:   0%|          | 0/512 [00:00<?, ?it/s]

Processing coco_part: 100%|██████████| 100/100 [00:52<00:00,  1.90it/s]



CLIP Score on coco_part (generated captions): 0.3140
Average caption generation time: 0.4986 seconds

Evaluating CLIP Score on anime with generated captions...

Processing category: anime


Resolving data files:   0%|          | 0/3004 [00:00<?, ?it/s]

...

Processing cartoon: 100%|██████████| 100/100 [00:49<00:00,  2.01it/s]



CLIP Score on cartoon (generated captions): 0.3215
Average caption generation time: 0.4728 seconds

Evaluating CLIP Score on human with generated captions...

Processing category: human


Resolving data files:   0%|          | 0/3000 [00:00<?, ?it/s]

Processing human: 100%|██████████| 100/100 [00:56<00:00,  1.79it/s]


CLIP Score on human (generated captions): 0.2930
Average caption generation time: 0.4932 seconds

Final CLIP Scores:
CLIP Score on dataset coco: 0.3140
Average time per image on dataset coco: 0.4986
CLIP Score on dataset anime: 0.2587
Average time per image on dataset anime: 0.4583
CLIP Score on dataset cartoon: 0.3215
Average time per image on dataset cartoon: 0.4728
CLIP Score on dataset human: 0.2930
Average time per image on dataset human: 0.4932


## Blip-Large

In [9]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

gen_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
gen_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [12]:
def generate_caption(image):
    # unconditional image captioning
    inputs = gen_processor(image, return_tensors="pt").to("cuda")
    
    out = gen_model.generate(**inputs)
    caption = gen_processor.decode(out[0], skip_special_tokens=True)

    return caption

In [13]:
if __name__ == "__main__":
    results = {}
    for dataset_name, dataset_info in DATASETS.items():
        score, average_time = compute_clip_score_with_generated(clip_model, clip_preprocess, dataset_dirs=dataset_info['dataset_dir'], dataset_name=dataset_info['name'])
        results[dataset_name] = (score, average_time)

    print("\nFinal CLIP Scores:")
    for dataset_name, result in results.items():
        print(f"CLIP Score on dataset {dataset_name}: {result[0]:.4f}")
        print(f'Average time per image on dataset {dataset_name}: {result[1]:.4f}')


Evaluating CLIP Score on coco_part with generated captions...

Processing category: coco_part


Resolving data files:   0%|          | 0/512 [00:00<?, ?it/s]

Processing coco_part: 100%|██████████| 100/100 [00:36<00:00,  2.78it/s]



CLIP Score on coco_part (generated captions): 0.2937
Average caption generation time: 0.3245 seconds

Evaluating CLIP Score on anime with generated captions...

Processing category: anime


Resolving data files:   0%|          | 0/3004 [00:00<?, ?it/s]

...


CLIP Score on cartoon (generated captions): 0.2974
Average caption generation time: 0.3036 seconds

Evaluating CLIP Score on human with generated captions...

Processing category: human


Resolving data files:   0%|          | 0/3000 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Processing human: 100%|██████████| 100/100 [00:40<00:00,  2.46it/s]


CLIP Score on human (generated captions): 0.2755
Average caption generation time: 0.3208 seconds

Final CLIP Scores:
CLIP Score on dataset coco: 0.2937
Average time per image on dataset coco: 0.3245
CLIP Score on dataset anime: 0.2561
Average time per image on dataset anime: 0.3105
CLIP Score on dataset cartoon: 0.2974
Average time per image on dataset cartoon: 0.3036
CLIP Score on dataset human: 0.2755
Average time per image on dataset human: 0.3208
